## Bottle框架简介
* Bottle的基本设计理念
    * URL 映射 : 将我们的URL映射到具体的函数上，实现处理
    * 模板引擎
    * Utiles : 基础功能，访问以及修改对应的表单数据，文件，cookies，HTTP头等元数据
    * Server

## Bottle路由
Bottle中的开发理念是使用路由route将所有的请求都绑定到对应的函数上，对请求的解析都是使用函数完成

## Bottle模板
内置的模板引擎可以使用
    * template()
    * view()
两种函数实现模板HTML渲染工作，我们只需要提供**模板的名字**和**提供给模板的变量**即可  
模板存储的路径可以使用模块内的列表变量**bottle.TEMPLATE_PATH**(一般都是['.' , './view'] , 当然可以动态的修改)获得

1. 在bottle的py文件下，我们创建对应的.tpl的html模板文件(['.' , './views']下都可以自动找到)，我们只需要使用函数template制定模板文件的名称和对应的使用需要的变量就可以

2. 模板参数传递
    * 在.tpl文件中定义我们的变量的名称，变量使用
        ```{{}}```
      的方式标明
    * 在py文件中，我们在template函数的之后的参数中使用字典传递方式传递对应的参数
    ```
    template('template_name(可以不用加tpl后缀)' , tname = name , tage = age)
    ```
3. view 的使用方式和参数的传递
    view 用做装饰器，参数是指定的模板文件的名称，函数的返回值是字典型的参数
４． 关于我们的模板文件的一些说明
    * 模板文件中{{var}}制定变量
    * var可以是python的数据结构，并且可以使用python中的语法(列表的索引，字典的获取函数...)
    * 只要在 {{...}} 中的 Python 语句返回一个字符串或有一个字符串的表达形式，它就是一个有效的语句。

## 模板具体语法
* 嵌入python代码
    * 一行使用%开头代表这一行是一个python代码块，我们对这种类python的代码只需要在末尾显示的添加%end来申明代码块的结束  
    * 没有用%开头的行一行普通的html文本渲染

```
<html>
  <head><title>登录页面</title></head>
  <body><p><h2>管理员登录</h2></p>
    <p><h2>会员信息</h2></p>
    %if tname:
    姓名:{{tname}}</br>
    %end
    <p></p>
    %if tnum:
    tnum:{{tnum}}</br>
    %else:
    tnum没有定义
    %end
    <p>{{get('tnum','666')}}在没有定义tnum的时候会用这句语句来进行赋值，定义了就不会执行</p>
    <p></p>
    %for i in tbook:
    {{i}}</br>
    %end
  </body>
</html>```

* 模板嵌套
    * %include('tpl_name.tpl')
        加载其他模板使用到当前的%include这里
    ```
    base.tpl
    <html>
      <head>
        <title>{{get('title','base.tpl')}}</title>
      </head>
      <body>
        <h2>h2</h2>
        <div>
          在这里加载一个memu.tpl模板文件(views文件夹后者当前目录下)
          %include('memu.tpl')
        </div>
      </body>
    </html>
    ```
    ```
    memu.tpl
    <ul>
      <li>python</li>
      <li>linux</li>
      <li>php</li>
      <li>c</li>
    </ul>
    ```

* 模板继承
    * %rebase tpl_name ...kargvs
        * 继承上层模板，将当前的模板嵌套到上层的模板中，还可以传递参数到上层的模板中，被加载的模板需要留下占位符准备嵌套其他的rebase的模板
        * tpl_name是被集成的tpl文件的名称可以不用加tpl后缀
        * 之后的kargvs是传递给我们之前的被继承的tpl文件的参数，用逗号分割
    * 一般都是写一个最底层的模板等待被其他的模板继承使用，减少重复代码，将层次结构类似的HTML代码整合起来，形成继承关系代码复用
    ```
    这里我们集成上面的base.tpl模板文件
    <p>%rebase base title='替换原来的linux',name='lantian'
    ```

## 静态资源
1. **static_file函数可以用来家在本地的文件到我们的浏览器上**
2. 流程
    * 使用动态路由匹配对应的文件（这里的路由使用src的属性发起的）
    * static_file（filename , path）加载对应的文件到浏览器上
        * filename是动态路由的动态文件名
        * path是我们制定的文件名所在的目录的绝对路径

In [2]:
'''
index.tpl
<html>
  <body>
    这里的路由跳转到/images/stars.jpeg，启动我们的server函数执行返回本机图片资源
    <img src='/images/stars.jpeg'>
  </body>
</html>
'''
from bottle import static_file , route , template , run

@route('/images/<filename:re:.*\.jpeg>')    # 这里的正则表达是的使用技巧是可以使用|来制定多种后缀的资源文件
def server(filename):
    return static_file(filename , 'path on the computer for the photo of .jpeg')

@route('/')    # 根的路径匹配
def index():
    return template('index')    # 使用index的模板

run(host = '127.0.0.1' , port = '8888')

## 文件下载
1. 对于函数**static_file('...' , '...' , download = '...')**
2. 在当前的醒目的py文件同目录下创建一个资源目录download用来存储资源文件，在html中使用<a>,href属性路由指向我们的server函数点击可以执行下载

In [3]:
#!/usr/bin/python3

from bottle import route , run
from bottle import template , view , static_file

@route('/images/<filename:re:.*\.jpeg>')
def server(filename):
    return static_file(filename , './download' , download = filename)

@route('/')
def login():
    return template('photo')

# run(host = '127.0.0.8' , port = 8888)